In [7]:
import json
import ollama
import openai

client = openai.OpenAI()

In [8]:
SYSTEM_PROMPT = ""
with open("../../global_system_prompt.txt", "r", encoding="utf-8") as f:
    SYSTEM_PROMPT = f.read()
VALIDATE_PROMPT = ""
with open("../../validity_check_prompt.txt", "r", encoding="utf-8") as f:
    VALIDATE_PROMPT = f.read()

In [9]:
from collections import Counter

def get_structure(json_data):
    try:
        data = json.loads(json_data)
        return {
            "atoms_count": len(data.get("atoms", [])),
            "rules_count": len(data.get("rules", [])),
            "strict_rules": len([r for r in data.get("rules", []) if r.get("type") == "strict"]),
            "defeasible_rules": len([r for r in data.get("rules", []) if r.get("type") == "defeasible"]),
            "attacks_count": len(data.get("attacks", [])),
            "attack_types": Counter([a.get("type") for a in data.get("attacks", [])])
        }
    except:
        return "Invalid JSON"

In [10]:
def respond(text: str, system_prompt: str):
    response = client.responses.create(
        model="gpt-4o-mini",
        temperature=0,
        input=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": text,
            }
        ],
        text={
            "format": {
                "type": "json_schema",
                "name": "argument_framework",
                "strict": True,
                "schema": {
                    "type": "object",
                    "properties": {
                        "atoms": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "id": {"type": "string"},
                                    "text": {"type": "string"}
                                },
                                "required": ["id", "text"],
                                "additionalProperties": False
                            }
                        },
                        "rules": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "id": {"type": "string"},
                                    "type": {
                                        "type": "string",
                                        "enum": ["strict", "defeasible"]
                                    },
                                    "premises": {
                                        "type": "array",
                                        "items": {"type": "string"}
                                    },
                                    "conclusion": {"type": "string"}
                                },
                                "required": ["id", "type", "premises", "conclusion"],
                                "additionalProperties": False
                            }
                        },
                        "arguments": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "id": {"type": "string"},
                                    "premises": {
                                        "type": "array",
                                        "items": {"type": "string"}
                                    },
                                    "applied_rules": {
                                        "type": "array",
                                        "items": {"type": "string"}
                                    },
                                    "conclusion": {"type": "string"},
                                    "sub_arguments": {
                                        "type": "array",
                                        "items": {"type": "string"}
                                    }
                                },
                                "required": ["id", "premises", "applied_rules", "conclusion", "sub_arguments"],
                                "additionalProperties": False
                            }
                        },
                        "attacks": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "attacker": {"type": "string"},
                                    "target": {"type": "string"},
                                    "type": {
                                        "type": "string",
                                        "enum": ["rebut", "undercut"]
                                    }
                                },
                                "required": ["attacker", "target", "type"],
                                "additionalProperties": False
                            }
                        }
                    },
                    "required": ["atoms", "rules", "arguments", "attacks"],
                    "additionalProperties": False
                }
            }
        }
    )

    return response.output_text.strip()

In [11]:
prompts=[]
with open("prompts.txt", "r") as f:
    prompts = f.read()
for i in prompts.split("@"):
    print("Prompt:", i)
    primary_output = respond(i, SYSTEM_PROMPT)
    print("Primary model response:\n",primary_output)
    validated_output = respond(primary_output, VALIDATE_PROMPT)
    print("Validation response:\n",validated_output)
    

Prompt: If a creature is a bird, it has wings.  
This creature has wings.  
Therefore, this creature is a bird.

Primary model response:
 {
  "atoms": [
    {"id": "a1", "text": "If a creature is a bird, it has wings"},
    {"id": "a2", "text": "This creature has wings"},
    {"id": "a3", "text": "This creature is a bird"}
  ],
  "rules": [
    {"id": "r1", "type": "strict", "premises": ["a1"], "conclusion": "a2"}
  ],
  "arguments": [
    {"id": "arg1", "premises": ["a1", "a2"], "applied_rules": ["r1"], "conclusion": "a3", "sub_arguments": []}
  ],
  "attacks": []
}
Validation response:
 {
  "atoms": [
    {"id": "a1", "text": "If a creature is a bird, it has wings"},
    {"id": "a2", "text": "This creature has wings"},
    {"id": "a3", "text": "This creature is a bird"}
  ],
  "rules": [
    {"id": "r1", "type": "strict", "premises": ["a1"], "conclusion": "a2"},
    {"id": "r2", "type": "strict", "premises": ["a3"], "conclusion": "a2"}
  ],
  "arguments": [
    {"id": "arg1", "premis